# Write generated events as a ROOT file

impy comes with writers that write the event record in different formats. Two popular formats are HepMC3 and ROOT. ROOT output is similar but not identical to the CRMC ROOT output. CERN's ROOT framework is not required to write ROOT files, only the comparably small uproot Python library.

Note: that you can also use the command-line interface to generate events in a specific format, so running the code in this notebook is not really needed. You could use this notebook as a template, however, if you want to apply some complex filtering before saving the event.

In [1]:
import uproot
from impy.models import Pythia8
from impy.writer import Root
from impy.kinematics import CenterOfMass
from impy.constants import GeV
import numpy as np
from particle import literals as lp

In [2]:
kin = CenterOfMass(10 * GeV, "proton", "proton")
model = Pythia8(kin, seed=2)


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.307      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 25 Feb 2022  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 15 Dec 2022 at 17:07:34    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

In [3]:
nevents = 1000

with Root("output.root", model) as f:
    for event in model(nevents):
        event = event.without_parton_shower()
        event = event[2:]  # cut away beam particles
        f.write(event)

Now we are doing generating and saving. The code below demonstrates how to read the file back with uproot.

In [8]:
import numpy as np
import uproot
from particle import literals as lp
import numba as nb


def pt_eta(px, py, pz):
    pt = np.sqrt(px ** 2 + py ** 2)
    eta = np.arcsinh(pz / pt)
    return pt, eta

photon_pid = int(lp.photon.pdgid)
pi0_pid = int(lp.pi_0.pdgid)


@nb.njit
def select_photons_from_pi0_decay(pid, par):
    result = np.zeros(len(pid), dtype="bool")
    for i, pidi in enumerate(pid):
        if pidi != photon_pid:
            continue
        if par[i] < 0:
            continue
        parent_pid = pid[par[i]]
        if parent_pid != pi0_pid:
            continue
        result[i] = True
    return result

with uproot.open("output.root") as f:
    tree = f["event"]
    branches = ["px", "py", "pz", "m", "pdgid", "status", "parent"]
    # loop over chunks in ROOT tree
    for chunk in tree.iterate(branches):
        # loop over individual events
        for px, py, pz, m, pid, sta, par in zip(*(chunk[k] for k in branches)):
            pt, eta = pt_eta(px, py, pz)

            print("all particles")
            print("  pseudorapidity", eta)
            print("  pdgid", pid)

            # select only long-lived particles
            mask = sta == 1
            print("long-lived particles")
            print("  pseudorapidity", eta[mask])
            print("  pdgid", pid[mask])

            # select only photons from pi0 decays
            mask = select_photons_from_pi0_decay(pid, par)
            print("  photons from pi0 decay")
            print("  pseudorapidity", eta[mask])

            break  # remove to not stop after first event


all particles
  pseudorapidity [0.926, 3.44, 3.42, 1.28, 0.297, 1.49]
  pdgid [321, 3214, 3122, 111, 22, 22]
long-lived particles
  pseudorapidity [0.926, 3.42, 0.297, 1.49]
  pdgid [321, 3122, 22, 22]
  photons from pi0 decay
  pseudorapidity [0.297, 1.49]
